<a href="https://colab.research.google.com/github/MrSimple07/ArchitecturesNN_for_DL/blob/main/AbdurakhimovM_lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --quiet transformers==4.37.2 accelerate==0.24.0 sentencepiece==0.1.99 optimum==1.13.2 peft==0.5.0 bitsandbytes==0.41.2.post2 datasets==2.14.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 33.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm, trange
import torch
import torch.nn as nn
import torch.nn.functional as F
import peft

import transformers
from datasets import load_dataset

import random
const_seed = 100

In [3]:
torch.cuda.is_available()

True

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Part 0: Initializing the model and tokenizer

let's take mistral model for our experiments (https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2) that was tuned to follow user instructions. Pay attention that we load model in 4 bit to decrease the memory usage.

In [5]:
model_name = 'mistralai/Mistral-7B-Instruct-v0.2'


# load llama tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device)
tokenizer.pad_token_id = tokenizer.eos_token_id

# Note: to speed up inference you can use flash attention 2 (https://huggingface.co/docs/transformers/perf_infer_gpu_one#flashattention-2)
model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
    load_in_4bit=True, torch_dtype=torch.float32,  # weights are 4-bit; layernorms and activations are fp32
)
for param in model.parameters():
    param.requires_grad=False

model.gradient_checkpointing_enable()  # only store a small subset of activations, re-compute the rest.
model.enable_input_require_grads()     # override an implementation quirk in gradient checkpoints that disables backprop unless inputs require grad
# more on gradient checkpointing: https://pytorch.org/docs/stable/checkpoint.html https://arxiv.org/abs/1604.06174

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

# Part 1 (5 points): Prompt-engineering

**There are different strategies for text generation in huggingface:**

| Strategy | Description | Pros & Cons |
| --- | --- | --- |
| Greedy Search | Chooses the word with the highest probability as the next word in the sequence. | **Pros:** Simple and fast. <br> **Cons:** Can lead to repetitive and incoherent text. |
| Sampling with Temperature | Introduces randomness in the word selection. A higher temperature leads to more randomness. | **Pros:** Allows exploration and diverse output. <br> **Cons:** Higher temperatures can lead to nonsensical outputs. |
| Nucleus Sampling (Top-p Sampling) | Selects the next word from a truncated vocabulary, the "nucleus" of words that have a cumulative probability exceeding a pre-specified threshold (p). | **Pros:** Balances diversity and quality. <br> **Cons:** Setting an optimal 'p' can be tricky. |
| Beam Search | Explores multiple hypotheses (sequences of words) at each step, and keeps the 'k' most likely, where 'k' is the beam width. | **Pros:** Produces more reliable results than greedy search. <br> **Cons:** Can lack diversity and lead to generic responses. |
| Top-k Sampling | Randomly selects the next word from the top 'k' words with the highest probabilities. | **Pros:** Introduces randomness, increasing output diversity. <br> **Cons:** Random selection can sometimes lead to less coherent outputs. |
| Length Normalization | Prevents the model from favoring shorter sequences by dividing the log probabilities by the sequence length raised to some power. | **Pros:** Makes longer and potentially more informative sequences more likely. <br> **Cons:** Tuning the normalization factor can be difficult. |
| Stochastic Beam Search | Introduces randomness into the selection process of the 'k' hypotheses in beam search. | **Pros:** Increases diversity in the generated text. <br> **Cons:** The trade-off between diversity and quality can be tricky to manage. |
| Decoding with Minimum Bayes Risk (MBR) | Chooses the hypothesis (out of many) that minimizes expected loss under a loss function. | **Pros:** Optimizes the output according to a specific loss function. <br> **Cons:** Computationally more complex and requires a good loss function. |

Documentation references:
- [reference for `AutoModelForCausalLM.generate()`](https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/text_generation#transformers.GenerationMixin.generate)
- [reference for `AutoTokenizer.decode()`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.decode)
- Huggingface [docs on generation strategies](https://huggingface.co/docs/transformers/generation_strategies)

In [23]:
# TODO: create a function for generation with huggingface


def get_answer(tokenizer, model, messages, max_new_tokens=200,
               temperature=0.5, do_sample=True):
    input_message = " ".join(message["content"] for message in messages)

    inputs = tokenizer.encode(input_message, return_tensors="pt")

    inputs = inputs.to(model.device)

    outputs = model.generate(
        inputs,
        max_length=len(inputs[0]) + max_new_tokens,
        temperature=temperature,
        do_sample=do_sample,
        pad_token_id=tokenizer.eos_token_id,
        attention_mask=torch.ones_like(inputs)
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return decoded

In [ ]:
# Let's try our model

messages = [
    {"role": "user", "content": "Write an explanation of tensors for 5 year old"},
]

print(get_answer(tokenizer, model, messages))

Write an explanation of tensors for 5 year olds.

Tensors are like special kind of boxes. But not just any boxes, they can hold different things in different directions!

Imagine you have a box of toys. You can put different toys in it, like cars, blocks, or dolls. But a tensor box is not just one-dimensional, it can have more dimensions!

A two-dimensional tensor box could hold cars and blocks, but they can't mix. Cars go in one direction, and blocks go in another. A three-dimensional tensor box could hold cars, blocks, and dolls, and they can't mix in two directions!

Tensors help us understand things in the world that have more than one direction, like a picture, or the position of an object in space. They help us organize and understand complex information!


You should obtain an explanation from the model. If so, let us go further!

Now we will take a sample from boolQ (https://huggingface.co/datasets/google/boolq) dataset and try prompting techniques to extract the needed answer and calculate its quality

In [6]:
df = load_dataset("google/boolq")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/9427 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3270 [00:00<?, ? examples/s]

In [7]:
# Fixing 20 validation examples

random.seed(const_seed)
idx = random.sample(range(1, 3270), 20)

In [9]:
# sample you will work with
df_sample = df["validation"].select(idx)

In [14]:
df_sample

Dataset({
    features: ['question', 'answer', 'passage'],
    num_rows: 20
})

In [26]:
df = df_sample.to_pandas()

df[['question', 'answer', 'passage']].to_csv('dataset.csv', index=False)
df.shape

(20, 3)

In [ ]:
# For instance, you can construct your prompt the following way
messages = [
    {"role": "user", "content": '''You are given a text and question. Answer only "true" or "false".
text: As with other games in The Elder Scrolls series, the game is set on the continent of Tamriel. The events of the game occur a millennium before those of The Elder Scrolls V: Skyrim and around 800 years before The Elder Scrolls III: Morrowind and The Elder Scrolls IV: Oblivion. It has a broadly similar structure to Skyrim, with two separate conflicts progressing at the same time, one with the fate of the world in the balance, and one where the prize is supreme power on Tamriel. In The Elder Scrolls Online, the first struggle is against the Daedric Prince Molag Bal, who is attempting to meld the plane of Mundus with his realm of Coldharbour, and the second is to capture the vacant imperial throne, contested by three alliances of the mortal races. The player character has been sacrificed to Molag Bal, and Molag Bal has stolen their soul, the recovery of which is the primary game objective.
question: is elder scrolls online the same as skyrim
answer: '''},
]

print(get_answer(tokenizer, model, messages)[0])

Y


In [24]:
messages = [
    {"role": "user", "content": '''You are given a text and question. Answer only "true" or "false".
text: As with other games in The Elder Scrolls series, the game is set on the continent of Tamriel. The events of the game occur a millennium before those of The Elder Scrolls V: Skyrim and around 800 years before The Elder Scrolls III: Morrowind and The Elder Scrolls IV: Oblivion. It has a broadly similar structure to Skyrim, with two separate conflicts progressing at the same time, one with the fate of the world in the balance, and one where the prize is supreme power on Tamriel. In The Elder Scrolls Online, the first struggle is against the Daedric Prince Molag Bal, who is attempting to meld the plane of Mundus with his realm of Coldharbour, and the second is to capture the vacant imperial throne, contested by three alliances of the mortal races. The player character has been sacrificed to Molag Bal, and Molag Bal has stolen their soul, the recovery of which is the primary game objective.
question: is elder scrolls online the same as skyrim
answer: '''},
]

print(get_answer(tokenizer, model, messages))

You are given a text and question. Answer only "true" or "false".
text: As with other games in The Elder Scrolls series, the game is set on the continent of Tamriel. The events of the game occur a millennium before those of The Elder Scrolls V: Skyrim and around 800 years before The Elder Scrolls III: Morrowind and The Elder Scrolls IV: Oblivion. It has a broadly similar structure to Skyrim, with two separate conflicts progressing at the same time, one with the fate of the world in the balance, and one where the prize is supreme power on Tamriel. In The Elder Scrolls Online, the first struggle is against the Daedric Prince Molag Bal, who is attempting to meld the plane of Mundus with his realm of Coldharbour, and the second is to capture the vacant imperial throne, contested by three alliances of the mortal races. The player character has been sacrificed to Molag Bal, and Molag Bal has stolen their soul, the recovery of which is the primary game objective.
question: is elder scrolls on

In [ ]:
messages = [
    {"role": "user", "content": '''You are given a text and question. Answer only "true" or "false".
text: As with other games in The Elder Scrolls series, the game is set on the continent of Tamriel. The events of the game occur a millennium before those of The Elder Scrolls V: Skyrim and around 800 years before The Elder Scrolls III: Morrowind and The Elder Scrolls IV: Oblivion. It has a broadly similar structure to Skyrim, with two separate conflicts progressing at the same time, one with the fate of the world in the balance, and one where the prize is supreme power on Tamriel. In The Elder Scrolls Online, the first struggle is against the Daedric Prince Molag Bal, who is attempting to meld the plane of Mundus with his realm of Coldharbour, and the second is to capture the vacant imperial throne, contested by three alliances of the mortal races. The player character has been sacrificed to Molag Bal, and Molag Bal has stolen their soul, the recovery of which is the primary game objective.
question: is elder scrolls online the same as skyrim
answer: '''},
]

In [36]:
messages = [
    {"role": "user", "content": '''You are given a text and question. Answer only "true" or "false".
text: "As the Senate president, the vice president presides over its deliberations (or delegates this task to a member of the Senate), but is allowed to vote only when it is necessary to break a tie. While this vote-casting prerogative has been exercised chiefly on legislative issues, it has also been used to break ties on the election of Senate officers, as well as on the appointment of Senate committees. In this capacity, the vice president also presides over joint sessions of Congress.",
question: is the vice president the head of the senate
answer: '''},
    {"role": "user", "content": '''You are given a text and question. Answer only "true" or "false".
text: "The Federal Reserve began taking high-denomination currency out of circulation (destroying large bills received by banks) in 1969. As of May 30, 2009, only 336 $10,000 bills were known to exist; 342 remaining $5,000 bills; and 165,372 remaining $1,000 bills. Due to their rarity, collectors often pay considerably more than the face value of the bills to acquire them. Some are in museums in other parts of the world.",
question: can i get $1 000 bill from the bank
answer: '''},
    {"role": "user", "content": '''You are given a text and question. Answer only "true" or "false".
text: "The ground squirrels are members of the squirrel family of rodents (Sciuridae) which generally live on or in the ground, rather than trees. The term is most often used for the medium-sized ground squirrels, as the larger ones are more commonly known as marmots (genus Marmota) or prairie dogs, while the smaller and less bushy-tailed ground squirrels tend to be known as chipmunks. Together, they make up the ``marmot tribe'' of squirrels, Marmotini, and the large and mainly ground squirrel subfamily Xerinae, and containing six living genera. Well-known members of this largely Holarctic group are the marmots (Marmota), including the American groundhog, the chipmunks, the susliks (Spermophilus), and the prairie dogs (Cynomys). They are highly variable in size and habitus, but most are remarkably able to rise up on their hind legs and stand fully erect comfortably for prolonged periods. They also tend to be far more gregarious than other squirrels, and many live in colonies with complex social structures. Most Marmotini are rather short-tailed and large squirrels, and the alpine marmot (Marmota marmota) is the largest living member of the Sciuridae, at 53--73 cm in length and weighing 5--8 kg.",
question: is a chipmunk the same as a ground squirrel
answer: '''},
    {"role": "user", "content": '''text: Kim Garner, the senior vice president of marketing and artist development at Universal Republic Records, said that Brand and Universal Pictures ``felt very strongly about doing something like this as opposed to a traditional soundtrack,'' and that they ``wanted to release it like we would an actual rock band's album.'',
question: is russell brand singing in get him to the greek
answer: '''},
    {"role": "user", "content": '''text: Compared to similar technology in other sports, goal-line technology is a relatively recent addition to association football; its integration having been opposed by the sport's authorities. In July 2012, the International Football Association Board (IFAB) officially approved the use of goal line technology, amending the Laws of the Game to permit (but not require) its use. Due to its expense, goal-line technology is only used at the highest levels of the game. Goal-line technology is currently used in the top European domestic leagues, and at major international competitions such as the 2014 Men's, 2018 Men's and 2015 Women's FIFA World Cups.,
question: is there goal line technology in the world cup
answer: '''},
    {"role": "user", "content": '''text: Compared to similar technology in other sports, goal-line technology is a relatively recent addition to association football; its integration having been opposed by the sport's authorities. In July 2012, the International Football Association Board (IFAB) officially approved the use of goal line technology, amending the Laws of the Game to permit (but not require) its use. Due to its expense, goal-line technology is only used at the highest levels of the game. Goal-line technology is currently used in the top European domestic leagues, and at major international competitions such as the 2014 Men's, 2018 Men's and 2015 Women's FIFA World Cups.,
question: is there goal line technology in the world cup
answer: '''},
    {"role": "user", "content": '''text: The Boss Baby: Back in Business is an American computer-animated web television series produced by DreamWorks Animation that is a follow-up of the 2017 film The Boss Baby, loosely based on the book of the same name by Marla Frazee. The series premiered on Netflix on April 6, 2018.,
question: is boss baby back in business on netflix
answer: '''},
    {"role": "user", "content": '''text: Baby back ribs (also back ribs or loin ribs) are taken from the top of the rib cage between the spine and the spare ribs, below the loin muscle. They have meat between the bones and on top of the bones, and are shorter, curved, and sometimes meatier than spare ribs. The rack is shorter at one end, due to the natural tapering of a pig's rib cage. The shortest bones are typically only about 3 in (7.6 cm) and the longest is usually about 6 in (15 cm), depending on the size of the hog. A pig side has 15 to 16 ribs (depending on the breed), but usually two or three are left on the shoulder when it is separated from the loin. So, a rack of back ribs contains a minimum of eight ribs (some may be trimmed if damaged), but can include up to 13 ribs, depending on how it has been prepared by the butcher. A typical commercial rack has 10--13 bones. If fewer than 10 bones are present, butchers call them ``cheater racks''.
question: are back ribs same as baby back ribs
answer: '''},
    {"role": "user", "content": '''text: The climate in the region is generally cool, owing to the southern latitude. There are no weather stations in the group of islands including Cape Horn; but a study in 1882--1883, found an annual rainfall of 1,357 millimetres (53.4 inches), with an average annual temperature of 5.2 °C (41.4 °F). Winds were reported to average 30 kilometres per hour (8.33 m/s; 18.64 mph), (5 Bf), with squalls of over 100 kilometres per hour (27.78 m/s; 62.14 mph), (10 Bf) occurring in all seasons. There are 278 days of rainfall (70 days snow) and 2,000 millimetres (79 inches) of annual rainfall,
question: is the southern tip of south america cold
answer: '''},
    {"role": "user", "content": '''text: The climate in the region is generally cool, owing to the southern latitude. There are no weather stations in the group of islands including Cape Horn; but a study in 1882--1883, found an annual rainfall of 1,357 millimetres (53.4 inches), with an average annual temperature of 5.2 °C (41.4 °F). Winds were reported to average 30 kilometres per hour (8.33 m/s; 18.64 mph), (5 Bf), with squalls of over 100 kilometres per hour (27.78 m/s; 62.14 mph), (10 Bf) occurring in all seasons. There are 278 days of rainfall (70 days snow) and 2,000 millimetres (79 inches) of annual rainfall,
question: is the southern tip of south america cold
answer: '''},
    {"role": "user", "content": '''text: The away goals rule is applied in many football competitions that involve two-leg fixtures, including the knockout stages of the UEFA Champions League, UEFA Europa League, CAF Champions League, CAF Confederation Cup and any two-legged playoffs in qualification for the FIFA World Cup or European Championships. Major League Soccer in the U.S. and Canada introduced the away goals rule in the MLS Cup Playoffs, in which the conference semifinals and finals (the quarterfinals and semifinals of the overall tournament) are two-legged, for the first time in 2014. The rule was first applied in this competition when the Seattle Sounders defeated FC Dallas in the 2014 Western Conference Semifinals.
question: do away goals count in the league playoffs
answer: '''},
    {"role": "user", "content": '''text: Nigella sativa (black caraway, also known as black cumin, nigella, and kalonji) is an annual flowering plant in the family Ranunculaceae, native to south and southwest Asia.
question: is black cumin seed same as black seed
answer: '''},
    {"role": "user", "content": '''text: Belgium have appeared in the finals tournament of the FIFA World Cup on 13 occasions, the first being at the first FIFA World Cup in 1930 where they finished in 11th place. The inaugural FIFA World Cup final was officiated by Belgian referee John Langenus.
question: have belgium ever reached the world cup final
answer: '''},
    {"role": "user", "content": '''text: In 2003, the United States withdrew remaining non-training troops or armament purchase support from Saudi Arabia, with 200 of these support personnel remaining, primarily at Eskan Village, a base which is owned by the Saudi Arabian government itself, in support of the US Military Training Mission (USMTM) in Saudi Arabia and the US Office of Program Management for the Saudi Arabian National Guard (OPM-SANG).
question: does us have military bases in saudi arabia
answer: '''},
    {"role": "user", "content": '''text: Each legislator shall be at least twenty-one years of age, an elector and resident of the District from which elected and shall have resided in the state for a period of two years prior to election.
question: does the florida constitution give a minimum age for legislators
answer: '''},
    {"role": "user", "content": '''text: Brie (/briː/; French: (bʁi)) is a soft cow's-milk cheese named after Brie, the French region from which it originated (roughly corresponding to the modern département of Seine-et-Marne). It is pale in color with a slight grayish tinge under a rind of white mould. The rind is typically eaten, with its flavor depending largely upon the ingredients used and its manufacturing environment.
question: is the white cover on brie cheese edible
answer: '''},
    {"role": "user", "content": '''text: In Australia, each state has its own constitution. Each state constitution preceded the Constitution of Australia as constitutions of the then separate British colonies, but all the states ceded powers to the Parliament of Australia as part of federation in 1901.
question: does each state have its own constitution australia
answer: '''},
    {"role": "user", "content": '''text: A table may have multiple foreign keys, and each foreign key can have a different parent table. Each foreign key is enforced independently by the database system. Therefore, cascading relationships between tables can be established using foreign keys.
question: can i have multiple foreign keys in a table
answer: '''},
    {"role": "user", "content": '''text: Delay of game is a penalty in ice hockey. It results in the offending player spending two minutes in the penalty box. In the NHL, delay of game is usually called under seven circumstances:
question: is there a delay of game in hockey
answer: '''},
    {"role": "user", "content": '''text: Since 22 February 2017, New Hampshire is a constitutional carry state, requiring no license to open carry or concealed carry a firearm in public. Concealed carry permits are still issued for purposes of reciprocity with other states.
question: do you need a permit to carry a gun in nh
answer: '''},
    {"role": "user", "content": '''text: On October 9, 2015, Los Lobos was nominated for induction into the Rock and Roll Hall of Fame for the first time.
question: is los lobos in the rock and roll hall of fame
answer: '''}
]

print(get_answer(tokenizer, model, messages))

You are given a text and question. Answer only "true" or "false".
text: "As the Senate president, the vice president presides over its deliberations (or delegates this task to a member of the Senate), but is allowed to vote only when it is necessary to break a tie. While this vote-casting prerogative has been exercised chiefly on legislative issues, it has also been used to break ties on the election of Senate officers, as well as on the appointment of Senate committees. In this capacity, the vice president also presides over joint sessions of Congress.",
question: is the vice president the head of the senate
answer:  You are given a text and question. Answer only "true" or "false".
text: "The Federal Reserve began taking high-denomination currency out of circulation (destroying large bills received by banks) in 1969. As of May 30, 2009, only 336 $10,000 bills were known to exist; 342 remaining $5,000 bills; and 165,372 remaining $1,000 bills. Due to their rarity, collectors often pay 

Is anything wrong with the output? Now it is time for you to play around and try to come up with some better prompt.

In [ ]:
# TODO: create function to evaluate answers
# Note: you can adapt function for different answer structures,
# but you should be able to automatically extract the target "true" or "false" components


def evaluate_answers(true_answers, predictions):
    if len(true_answers) != len(predictions):
        raise ValueError("Lengths of true_answers and predictions must be the same.")

    score = 0

    for true_answer, pred_answer in zip(true_answers, predictions):
        if true_answer.lower() == pred_answer.lower():
            score += 1

    accuracy = score / len(true_answers)

    return accuracy

TODO: Try and compare "naive" prompting (your best hand-crafted variant), few-shot prompting (https://www.promptingguide.ai/techniques/fewshot) and chain-of-thought prompting (step-be-step thinking - https://www.promptingguide.ai/techniques/cot).

Save the generation results into separate csv files and do not forget to attach them to your homework.

In [ ]:
import csv
import random

random_text = """
The Earth is the third planet from the Sun and the only astronomical object known to harbor life. It is home to millions of species, including humans, who inhabit it. The Earth formed over 4.5 billion years ago and is the only planet known to support life.
"""

naive_prompt = f"""
You are given a text and question. Answer only "true" or "false".
text: {random_text}
question: Is the statement true or false?
answer:
"""

few_shot_prompt = f"""
text: {random_text}
question: Is the statement true or false?
Few-shot examples:
- True: The Earth is the third planet from the Sun.
- False: The Earth is the tenth planet from the Sun.
answer:
"""

chain_of_thought_prompt = f"""
You are given a text and a question. Answer only "true" or "false".
Consider the following steps and respond accordingly:
Step 1: Read the text about the Earth.
Step 2: Identify the statement about the planet's position from the Sun.
Step 3: Determine if the statement is true or false based on your knowledge.
...
question: Is the statement true or false?
answer:
"""

def save_to_csv(filename, generated_text):
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Generated Text'])
        for text in generated_text:
            writer.writerow([text])


print(get_answer(tokenizer, model, naive_prompt))
print(get_answer(tokenizer, model, few_shot_prompt))
print(get_answer(tokenizer, model, chain_of_thought_prompt))

naive_generated_text = [naive_prompt] * 5
few_shot_generated_text = [few_shot_prompt] * 5
chain_of_thought_generated_text = [chain_of_thought_prompt] * 5

save_to_csv('naive_prompt_results.csv', naive_generated_text)
save_to_csv('few_shot_prompt_results.csv', few_shot_generated_text)
save_to_csv('chain_of_thought_prompt_results.csv', chain_of_thought_generated_text)


TypeError: string indices must be integers

# Part 2 (5 points): Fine-tuning with PEFT and LoRA

In [ ]:
peft_config = peft.PromptTuningConfig(task_type=peft.TaskType.CAUSAL_LM,
                                      num_virtual_tokens=16) #
model = peft.get_peft_model(model, peft_config)  # note: for most peft methods, this line also modifies model in-plac)))

In [ ]:
model.print_trainable_parameters() # Wow so small amount of trainable params

In [ ]:
# creating simple prompt formating
def format_prompt(sample):
    return f'''
    text: {sample['passage']}
    question: {sample['question']}
    answer: {sample['answer']}
    '''

TODO: initialize Trainer and pass train part of our dataset for 2-3 epoches

Note: carefully set max_seq_length and args (that are transformers.TrainingArguments)

TODO: save and check your tuned model. Provide scores on our 20 validation examples and save result to csv file